# DenseNet
upload transposed2.Rds

In [1]:
import rpy2.robjects as ro
from rpy2.robjects import r
import torch
from rpy2.robjects import pandas2ri
from torch.utils.data import Dataset, DataLoader
from torchvision import models
from torch import nn
from rpy2.robjects.conversion import localconverter

torch.manual_seed(88888888)

DATA_PATH = 'transposed2.Rds' #'/home/jgburk/PycharmProjects/IntroductionToDeepLearning/transposed.Rds'
NUM_FEATURES = 1000
NUM_CLASSES = 7
NUM_TRAIN_IT = 1000

BATCH_SIZE = 32


class PandasDataset(Dataset):

    def __init__(self, rds_fn):
      with localconverter(ro.default_converter + pandas2ri.converter):
        df = ro.conversion.rpy2py(r.readRDS(rds_fn))

        features = df.iloc[:, 0:NUM_FEATURES].values
        targets = df['sample_type'].values

        self.x = torch.tensor(features)
        self.y = torch.tensor(targets - 1)

    def __len__(self):
        return len(self.y)

    def __getitem__(self, i):
        return self.x[i], self.y[i]


def train(loader, dv):
    model.train()

    correct = 0
    for batch in loader:  # Iterate in batches over the training dataset.
        x = batch[0].reshape(BATCH_SIZE, 4, 10, 25).to(dv)
        y = batch[1].to(dv)
        out = model(x.float())  # Perform a single forward pass.
        y = torch.squeeze(y)
        loss = criterion(out, y.long())  # Compute the loss.
        loss.backward()  # Derive gradients.
        optimizer.step()  # Update parameters based on gradients.
        optimizer.zero_grad()  # Clear gradients.
        pred = out.argmax(dim=1)  # Use the class with highest probability.
        correct += int((pred == y).sum())  # Check against ground-truth labels.
    return correct / len(loader.dataset)  # Derive ratio of correct predictions.


rds_dataset = PandasDataset(DATA_PATH)
training_loader = DataLoader(rds_dataset, batch_size=BATCH_SIZE, shuffle=False)

model = models.DenseNet(num_classes=NUM_CLASSES)
conv0 = model.features.conv0
model.features.conv0 = nn.Conv2d(4, out_channels=conv0.out_channels, kernel_size=conv0.kernel_size,
                                 stride=conv0.stride, padding=conv0.padding, bias=conv0.bias)
model.features.transition1.pool = nn.AvgPool2d(kernel_size=2, stride=2, padding=1)
model.features.transition2.pool = nn.AvgPool2d(kernel_size=2, stride=2, padding=1)
model.features.transition3.pool = nn.AvgPool2d(kernel_size=2, stride=2, padding=1)
device = torch.device('cuda')
model.to(device)

optimizer = torch.optim.AdamW(model.parameters())
criterion = torch.nn.CrossEntropyLoss()

for epoch in range(NUM_TRAIN_IT):
    train(training_loader, device)
    train_acc = train(training_loader, device)
    print(f'Epoch: {epoch}, Train Acc: {train_acc}')
    if train_acc == 1.0:
        break

Epoch: 0, Train Acc: 0.15625
Epoch: 1, Train Acc: 0.2569444444444444
Epoch: 2, Train Acc: 0.2673611111111111
Epoch: 3, Train Acc: 0.2604166666666667
Epoch: 4, Train Acc: 0.2881944444444444
Epoch: 5, Train Acc: 0.28125
Epoch: 6, Train Acc: 0.3506944444444444
Epoch: 7, Train Acc: 0.2986111111111111
Epoch: 8, Train Acc: 0.4340277777777778
Epoch: 9, Train Acc: 0.4722222222222222
Epoch: 10, Train Acc: 0.3506944444444444
Epoch: 11, Train Acc: 0.4652777777777778
Epoch: 12, Train Acc: 0.5659722222222222
Epoch: 13, Train Acc: 0.5277777777777778
Epoch: 14, Train Acc: 0.6319444444444444
Epoch: 15, Train Acc: 0.6736111111111112
Epoch: 16, Train Acc: 0.7673611111111112
Epoch: 17, Train Acc: 0.8055555555555556
Epoch: 18, Train Acc: 0.75
Epoch: 19, Train Acc: 0.6736111111111112
Epoch: 20, Train Acc: 0.8229166666666666
Epoch: 21, Train Acc: 0.8333333333333334
Epoch: 22, Train Acc: 0.8472222222222222
Epoch: 23, Train Acc: 0.7777777777777778
Epoch: 24, Train Acc: 0.8263888888888888
Epoch: 25, Train Acc: